In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census

# Census API Key
from config import api_key
c = Census(api_key, year=2017)

In [2]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_df = pd.DataFrame(census_data)
'''

census_data = c.acs5.get(('NAME', "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"),
          {'for': 'state:{}'.format(states.MD.fips)})'''

# Column Reordering
census_df = census_df.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})



In [3]:
# Add in Poverty Rate (Poverty Count / Population)
census_df["Poverty Rate"] = 100 * \
    census_df["Poverty Count"].astype(
        int) / census_df["Population"].astype(int)

# Final DataFrame
census_df = census_df[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate"]]

census_df["Zipcode"] = census_df["Zipcode"].astype(str)


In [4]:
zip_list = ['55418', '55411', '55405', '55401', '55408', '55414', '55403',
       '55402', '55415', '55404', '55412', '55454', '55406', '55413',
       '55409', '55416', '55419', '55407', '55455', '55422', '55417',
       '55410', '55430', '55487', '55114', '55450', '55421', '55423',
       '55429']   #Hard coded....can get from ID_to_Zipcode program

In [5]:
zipcode_df = pd.DataFrame({"Zipcode": zip_list})


census_mpls_df = pd.merge(zipcode_df, census_df, on = 'Zipcode', how="left")

In [6]:
#List of downtown zipcodes
downtown = ['55401', '55402', '55403', '55404', '55405', '55411', '55415', '55454', '55487']

#Variable poverty rate criteria (currently set as MN's overall poverty rate)

poverty_rate_criteria = 10.5

#Build list to identify if zipcode's poverty rate is Low, High, or Downtown
for index, zip in census_mpls_df.iterrows():
 
    if zip["Zipcode"] in downtown:
        census_mpls_df.set_value(index, 'Poverty Type', 'Downtown')
        
    elif zip["Poverty Rate"] > poverty_rate_criteria: 
        census_mpls_df.set_value(index, 'Poverty Type', 'High')
        
    else:
        census_mpls_df.set_value(index, 'Poverty Type', 'Low')
                     
        
        

C:\Users\david\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  from ipykernel import kernelapp as app
C:\Users\david\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if sys.path[0] == '':
C:\Users\david\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


In [7]:
poverty_df = census_mpls_df[["Zipcode","Poverty Rate", "Poverty Type"]]

In [8]:
poverty_df.head()



,Zipcode,Poverty Rate,Poverty Type
0,55418,12.748436,High
1,55411,31.316104,Downtown
2,55405,14.252629,Downtown
3,55401,14.219141,Downtown
4,55408,15.597356,High


In [9]:
#export to csv
# Note to avoid any issues later, use encoding="utf-8"
poverty_df.to_csv("poverty_type.csv", encoding="utf-8", index=False)